In [ ]:
!pip install -q cassio datasets langchain openai tiktoken PyPDF2

In [ ]:
from datasets import load_dataset  
import cassio  
from langchain.llms import OpenAI  
from langchain.embeddings import OpenAIEmbeddings  
from langchain.vectorstores.cassandra import Cassandra  
from langchain.indexes.vectorstore import VectorStoreIndexWrapper  

In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
!pip install --upgrade astrapy

In [ ]:
from astrapy.db import AstraDB
db = AstraDB(
  token="",
  api_endpoint="")

print(f"Connected to Astra DB: {db.get_collections()}")

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = ""
ASTRA_DB_ID = ""
OPENAI_API_KEY = ""

In [ ]:
pdfreader = PdfReader('documents/budget_speech.pdf')

In [ ]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

In [ ]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headline." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\n Enter your question(or type 'quit' to exit): ").strip()
    else:
        query_text = input

In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat is your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER FOR : \"%s\"\n" % answer)

    print("SHOWING THE FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))